# Variational Quantum Eigensolver (VQE) with Classiq

The variational quantum eigensolver (VQE) is arguably the most common variational quantum algorithm. It combines quantum and classical techniques to solve optimization problems from industries such as finance, logistics, and chemistry. VQE works because of the variational principle, which defines the relationship between the lowest energy of a system and the expectation value of a given state, declaring a lower bound to the expectation value.

We are going to create a VQE algorithm to estimate the minimum energy(eigenvalue) of the following Hamiltonian
$$Z+0.1X$$ 
where $Z$ and $X$ are Pauli Operators and the ansatz we take is a single qubit $RY$ gate 

In [1]:
from typing import List

from classiq import *

#Defining the Hamiltonian from the problem
HAMILTONIAN = QConstant("HAMILTONIAN", List[PauliTerm], 
                        [            
            PauliTerm([Pauli.Z], 1),
            PauliTerm([Pauli.X], 0.1)])

#Defining the Ansatz for the Problem
@qfunc
def main(q: Output[QBit], angles: CArray[CReal, 1]) -> None:
    allocate(1, q)
    RY(angles[0], q) 

#Defining the Variational Quantum Eigensolver primitives with proper paramters
@cfunc
def cmain() -> None:
    res = vqe(
        HAMILTONIAN, #Hamiltonian of the problem
        False, #Maximize Parameter
        [],
        optimizer=Optimizer.COBYLA, # Classical Optimizer
        max_iteration=1000,
        tolerance=0.001,
        step_size=0,
        skip_compute_variance=False,
        alpha_cvar=1.0,
    )
    save({"result": res})

qmod = create_model(main, classical_execution_function=cmain)
qprog = synthesize(qmod)
write_qmod(qmod, name="vqe_primitives")

We can now model and get the results for the created vqe primitive:

In [2]:
show(qprog)

The list of other optimizers can be found here: [Link](https://docs.classiq.io/latest/reference-manual/built-in-algorithms/ground-state-solving/advanced-usage/solver-customization/)

In [3]:
estimation = execute(qprog)
# res.open_in_ide()
vqe_result = estimation.result()[0].value

/usr/bin/xdg-open: 882: x-www-browser: Permission denied
/usr/bin/xdg-open: 882: firefox: Permission denied
/usr/bin/xdg-open: 882: iceweasel: Permission denied
/usr/bin/xdg-open: 882: seamonkey: Permission denied
/usr/bin/xdg-open: 882: mozilla: Permission denied
/usr/bin/xdg-open: 882: epiphany: Permission denied
/usr/bin/xdg-open: 882: konqueror: Permission denied
/usr/bin/xdg-open: 882: chromium: Permission denied
/usr/bin/xdg-open: 882: chromium-browser: Permission denied
/usr/bin/xdg-open: 882: google-chrome: Permission denied
/usr/bin/xdg-open: 882: www-browser: Permission denied
/usr/bin/xdg-open: 882: links2: Permission denied
/usr/bin/xdg-open: 882: elinks: Permission denied
/usr/bin/xdg-open: 882: links: Permission denied
/usr/bin/xdg-open: 882: lynx: Permission denied
/usr/bin/xdg-open: 882: w3m: Permission denied
xdg-open: no method available for opening 'https://platform.classiq.io/circuit/b8d0e9eb-6b32-4825-a69c-614353c1b8b4?version=0.53.0'


In [4]:
print("Minimal energy of the Hamiltonian", vqe_result.energy)
print("Optimal parameters for the Ansatz", vqe_result.optimal_parameters)

Minimal energy of the Hamiltonian -1.0021484375
Optimal parameters for the Ansatz {'angles_param_0': -3.122778500085207}


### Mathematical Background

To find the minimal energy of the Hamiltonian 
$H = Z + 0.1X$ 
using the Variational Quantum Eigensolver (VQE) with an ansatz that consists of a simple 
$RY$ 
gate, we can follow these steps:

__First we define the Hamiltonian__:<br>
The Hamiltonian is given by:

$$
H = Z + 0.1X
$$

where 
$Z$ 
and 
$X$ 
are the Pauli-Z and Pauli-X operators, respectively.

__We can write the given Pauli matrices:__<br>
The Pauli-Z and Pauli-X matrices are:

$$
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}, \quad X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
$$

__Taking the Ansatz__:<br>
We use a single 
$RY$ 
gate as the ansatz. The 
$RY(\theta)$ 
gate is defined as:

$$
RY(\theta) = \begin{pmatrix} \cos\left(\frac{\theta}{2}\right) & -\sin\left(\frac{\theta}{2}\right) \\ \sin\left(\frac{\theta}{2}\right) & \cos\left(\frac{\theta}{2}\right) \end{pmatrix}
$$

The ansatz state can be written as:

$$
|\psi(\theta)\rangle = RY(\theta)|0\rangle
$$

We now proceed to the expectation value calculation:<br>
The goal is to find the expectation value of the Hamiltonian 
$H$ 
with respect to the ansatz state 
$|\psi(\theta)\rangle$:

$$
E(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle
$$

This involves calculating the expectation values of 
$Z$ 
and 
$X$ 
with respect to 
$|\psi(\theta)\rangle$.

__Calculations can be proceed__: <br>
The expectation value of 
$Z$ 
with respect to 
$|\psi(\theta)\rangle$ 
is:

$$
\langle \psi(\theta) | Z | \psi(\theta) \rangle = \cos(\theta)
$$

The expectation value of 
$X$ 
with respect to 
$|\psi(\theta)\rangle$ 
is:

$$
\langle \psi(\theta) | X | \psi(\theta) \rangle = \sin(\theta)
$$

Therefore, the total expectation value is:

$$
E(\theta) = \cos(\theta) + 0.1\sin(\theta)
$$

__Minimizing the Energy__:<br>
To find the minimal energy, we need to minimize 
$E(\theta)$ 
with respect to 
$\theta$. 
This can be done by solving the equation:

$$
\frac{dE(\theta)}{d\theta} = -\sin(\theta) + 0.1\cos(\theta) = 0
$$

Solving this gives:

$$
\tan(\theta) = 0.1
$$

$$
\theta = \tan^{-1}(0.1)
$$

__Minimal Energy__:<br>
Substitute 
$\theta$ 
back into the expression for 
$E(\theta)$:

$$
E_{\text{min}} = \cos(\tan^{-1}(0.1)) + 0.1\sin(\tan^{-1}(0.1))
$$

Using the identities:

$$
\cos(\tan^{-1}(x)) = \frac{1}{\sqrt{1+x^2}}, \quad \sin(\tan^{-1}(x)) = \frac{x}{\sqrt{1+x^2}}
$$

For 
$x = 0.1$:

$$
E_{\text{min}} = \frac{1}{\sqrt{1+0.01}} + 0.1 \cdot \frac{0.1}{\sqrt{1+0.01}} = \frac{1}{\sqrt{1.01}} + \frac{0.01}{\sqrt{1.01}} = \frac{1.01}{\sqrt{1.01}} = \sqrt{1.01}
$$

Thus, the minimal energy is:

$$
E_{\text{min}} = \sqrt{1.01}
$$

However, considering that the eigenvalues of 
$Z$ 
are 
$\pm 1$, 
the minimal energy will actually be:

$$
E_{\text{min}} = -\sqrt{1.01}
$$

Thus, the minimal energy is:

$$
E_{\text{min}} = -\sqrt{1.01} \approx -1.004987562112089
$$

__It is very similar to the result which we observed based on the VQE primitive in the Classiq simulator.__
